# **Bienvenidos a este agente especialista entre Oscar Ayala - Data Scientist y Python**

Este notebook implementa un flujo completo para modelamiento de series temporales usando ARIMA y SARIMA, automatizando la selección de parámetros y comparando métricas estadísticas.

In [ ]:
# ==============================
# 1. Instalación de librerías
# ==============================
!pip install pmdarima openpyxl

In [ ]:
# ==============================
# 2. Importación de librerías
# ==============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo

## **3. Simulación de datos con estacionalidad**

In [ ]:
fechas = pd.date_range(start='2015-01', periods=60, freq='M')
ingresos = 200000 + 5000*np.arange(60) + 30000*np.sin(2*np.pi*np.arange(60)/12) + np.random.normal(0, 20000, 60)
df = pd.DataFrame({'Fecha': fechas, 'Ingresos': ingresos})
df.set_index('Fecha', inplace=True)

# Visualización inicial
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Ingresos'], label='Ingresos')
plt.title('Ingresos Mensuales con Estacionalidad')
plt.xlabel('Fecha')
plt.ylabel('Ingresos')
plt.legend()
plt.show()

## **4. Selección automática ARIMA y SARIMA**

In [ ]:
# ARIMA
modelo_arima = auto_arima(df['Ingresos'], seasonal=False, stepwise=True, trace=True)
print(modelo_arima.summary())

# SARIMA
modelo_sarima = auto_arima(df['Ingresos'], seasonal=True, m=12, stepwise=True, trace=True)
print(modelo_sarima.summary())

## **5. Ajuste y predicciones futuras**

In [ ]:
modelo_arima.fit(df['Ingresos'])
modelo_sarima.fit(df['Ingresos'])

n_periods = 12
pred_arima, ci_arima = modelo_arima.predict(n_periods=n_periods, return_conf_int=True)
pred_sarima, ci_sarima = modelo_sarima.predict(n_periods=n_periods, return_conf_int=True)

fechas_futuras = pd.date_range(start=df.index[-1] + pd.offsets.MonthEnd(), periods=n_periods, freq='M')
pred_df_arima = pd.DataFrame({'Pred_ARIMA': pred_arima}, index=fechas_futuras)
pred_df_sarima = pd.DataFrame({'Pred_SARIMA': pred_sarima}, index=fechas_futuras)

## **6. Comparación de métricas (RMSE, MAE)**

In [ ]:
train = df.iloc[:-12]
test = df.iloc[-12:]

modelo_arima.fit(train['Ingresos'])
modelo_sarima.fit(train['Ingresos'])

pred_test_arima = modelo_arima.predict(n_periods=12)
pred_test_sarima = modelo_sarima.predict(n_periods=12)

rmse_arima = np.sqrt(mean_squared_error(test['Ingresos'], pred_test_arima))
rmse_sarima = np.sqrt(mean_squared_error(test['Ingresos'], pred_test_sarima))
mae_arima = mean_absolute_error(test['Ingresos'], pred_test_arima)
mae_sarima = mean_absolute_error(test['Ingresos'], pred_test_sarima)

print(f"ARIMA -> RMSE: {rmse_arima:.2f}, MAE: {mae_arima:.2f}")
print(f"SARIMA -> RMSE: {rmse_sarima:.2f}, MAE: {mae_sarima:.2f}")

## **7. Visualización comparativa ARIMA vs SARIMA**

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Ingresos'], label='Histórico')
plt.plot(pred_df_arima.index, pred_df_arima['Pred_ARIMA'], label='ARIMA', color='blue')
plt.plot(pred_df_sarima.index, pred_df_sarima['Pred_SARIMA'], label='SARIMA', color='green')
plt.title('Comparación Predicciones ARIMA vs SARIMA')
plt.xlabel('Fecha')
plt.ylabel('Ingresos')
plt.legend()
plt.show()

## **8. Exportación de métricas a Excel**

In [ ]:
wb = Workbook()
ws = wb.active
ws.title = "Resultados Modelos"
ws.append(["Modelo", "RMSE", "MAE"])
ws.append(["ARIMA", rmse_arima, mae_arima])
ws.append(["SARIMA", rmse_sarima, mae_sarima])

table = Table(displayName="TablaMetricas", ref="A1:C3")
style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
table.tableStyleInfo = style
ws.add_table(table)

wb.save("resultados_modelos.xlsx")